In [ ]:
import os 
from glob import glob
import pandas as pd

In [ ]:
# 특정 경로의 파일의 목록을 가져오는 기능 
# os 라이브러리를 이용
os.listdir('./')

In [ ]:
# glob 이용 
# 장점 : 파일의 경로와 파일의 이름을 하나의 리스트로 생성 
#       특정 확장자만 선택해서 리스트로 생성이 가능
json_list = glob("./*.json")

In [ ]:
# json_list를 이용하여 하나의 데이터프레임으로 단순 행 결합

# 빈 데이터프레임을 생성
total_df = pd.DataFrame()

for file_path in json_list:
    # print(file_path)
    df = pd.read_json(file_path)
    # total_df, df를 단순 행결합을 하여 total_df에 대입 
    total_df = pd.concat( [total_df, df], axis=0 )
    # print(df)
    # break
total_df.reset_index(drop=True, inplace=True)

In [ ]:
total_df.info()

In [ ]:
pd.concat(
    [ pd.read_json(file_path) for file_path in json_list[:5] ]
).info()

In [ ]:
# Aspects 의 데이터를 하나로 합치고 새로운 데이터 프레임을 생성 
aspect_df = pd.DataFrame(sum(total_df['Aspects'], []))

In [ ]:
aspect_df.info()

In [ ]:
# 데이터의 분균형 문제 확인 
aspect_df['SentimentPolarity'].value_counts()

In [ ]:
aspect_df.isna().sum()

In [ ]:
# 데이터셋에서 문자열의 좌우의 공백을 제거 
# 모든 컬럼이 Object 형이기 때문에 strip() 바로 사용 가능
aspect_df = aspect_df.map(lambda x : x.strip())

In [ ]:
aspect_df.isin(['']).sum()

In [ ]:
aspect_df['SentimentText'].value_counts()

In [ ]:
before_cnt = len(aspect_df)

aspect_df.drop_duplicates('SentimentText', inplace=True)

after_cnt = len(aspect_df)

print(f"제거가 된 행의 개수 {before_cnt - after_cnt}")

In [ ]:
# 1, 0, -1 의 비율을 확인 
aspect_df['SentimentPolarity'].value_counts()

In [ ]:
# 인덱스를 초기화 
aspect_df.reset_index(drop=True, inplace=True)

In [ ]:
# 토큰화 -> 백터화 
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import TfidfVectorizer

komoran = Komoran()
allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'MAG', 'SL']

def komoran_tokenize(text):
    tokens = []
    for word, pos in komoran.pos(text):
        if (pos in allow_pos) & (len(word) >= 2)  :
            tokens.append(word)
    return tokens

vectorizer = TfidfVectorizer(
    tokenizer= komoran_tokenize, 
    ngram_range=(1, 2), 
    min_df = 3, 
    max_df=0.8, 
    max_features=30000
)

In [ ]:
# 모델 생성 
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
svc = LinearSVC(random_state=42, class_weight='balanced')

multi_model = MultiOutputClassifier(svc)

pipe = Pipeline(
    [
        ('vector', vectorizer), 
        ('model', multi_model)
    ]
)


In [ ]:
# 계층화 폴드 
from sklearn.model_selection import KFold

skfold = KFold(n_splits=3, shuffle= True, 
                         random_state=42)

In [ ]:
aspect_df.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
aspect_df['Aspect'] = le.fit_transform(aspect_df['Aspect'])
aspect_df['SentimentPolarity'] = aspect_df[
    'SentimentPolarity'].astype('int')

In [ ]:
# 독립 변수 , 종속 변수 생성
X = aspect_df['SentimentText'].values
Y = aspect_df[['Aspect', 'SentimentPolarity']].values

In [ ]:
print(X.shape, Y.shape)

In [ ]:
print(type(Y[0][0]), type(Y[0][1]))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {
    'model__estimator__C' : [1.0, 2.0]
}
grid = GridSearchCV(
    estimator=pipe, 
    param_grid= params, 
    cv = skfold, 
    scoring="accuracy"
)

In [ ]:
grid.fit(X, Y)

In [ ]:
grid.best_

1. total_df에서 rawText 컬럼의 데이터들을 이용하여 Kkma를 이용하여 문장별로 나눠준다. 
2. grid의 best_estimator_에서 예측을 실행 
3. 실행된 결과 값을 이용하여 데이터프레임( RawText, Aspect_pred, Pola_pred )으로 생성 
4. rawText, Aspect_pred 값을 이용하여 그룹화 -> 그룹화 연산에는 평균 

In [ ]:
best_model = grid.best_estimator_

In [ ]:
total_df.columns

In [ ]:
total_df.loc[0, 'RawText']

In [ ]:
from konlpy.tag import Kkma

In [ ]:
kkma = Kkma()
# raw_list에는 리뷰 문단을 문장으로 나눈 리스트를 담기 위한 공간
raw_list = []
# raw_dict 리뷰 문단마다 index를 키값으로 value는 리뷰 문단
raw_dict = {}
for i in range(len(total_df)):
    # print(kkma.sentences(total_df.loc[i, 'RawText']))
    # break
    raw_list.append(kkma.sentences(total_df.loc[i, 'RawText']))
    raw_dict[i] = total_df.loc[i, 'RawText']

In [ ]:
sentence_df = pd.DataFrame()
for idx, raw in enumerate(raw_list):
    # print(raw)
    pred = best_model.predict(raw)
    # print(pred)
    temp_df = pd.DataFrame(pred, columns = [
        'Aspect_pred', 'Pola_pred'])
    temp_df['RawText'] = raw_dict[idx]
    # display(temp_df)
    sentence_df = pd.concat([sentence_df, temp_df])
    # break

In [ ]:
len(sum(raw_list, []))

In [ ]:
sentence_df

In [ ]:
group_df = sentence_df.groupby(['RawText', 
                                'Aspect_pred']).mean()

In [ ]:
group_df.reset_index(inplace=True)

In [ ]:
group_df['Aspect_pred'] = le.inverse_transform(group_df[
    'Aspect_pred'])

In [ ]:
group_df.index

In [ ]:
total_df.index

In [ ]:
# total_df와 group_df를 조인 결합 
review_df = pd.merge(total_df, group_df, on = 'RawText', how = 'inner')

In [ ]:
review_df.info()

In [ ]:
# ProductName의 빈도 수 체크 
len(review_df['ProductName'].unique())

In [ ]:
# 제품별 리뷰의 상세 감정 분석이 가능
# 제품 이름 중 가장 많은 리뷰를 가진 제품을 선택하여 감정 점수의 평균을 확인
review_df.groupby(['ProductName', 'RawText']).n()

In [ ]:
pd.pivot_table(
    data = review_df.drop_duplicates('RawText'),
    index = 'ProductName', 
    values = 'RawText', 
    aggfunc= ('count')
).sort_values('RawText',ascending=False).index[0]

In [ ]:
product_name = 'OO 코튼 가디건 '
# 해당 제품의 리뷰들의 전체적인 감정의 점수를 출력 
# case1 -> ProductName에서 필터링을 한 뒤 Aspect_pred 를 
# 기준으로  그룹화 -> Pola_pred의 평균 
test_df = review_df.loc[review_df['ProductName'] == product_name, ]
test_df

In [ ]:

test_df.groupby('Aspect_pred')['Pola_pred'].mean()

In [ ]:
# case2 -> review_df에서 ProductName과 Aspect_pred를 기준으로 그룹화
# Pola_pred의 평균을 구한다. 
# 원하는 제품명을 선택하여 확인 
group_df2 = review_df.groupby(['ProductName', 'Aspect_pred'])\
    ['Pola_pred'].mean()

In [231]:
group_df2[product_name]

Aspect_pred
가격      0.875000
기능      0.944444
길이      1.000000
두께      0.875000
디자인     1.000000
마감      1.000000
사이즈     0.300000
색상      0.950000
소재      1.000000
신축성     1.000000
제품구성   -1.000000
착용감     1.000000
촉감      1.000000
품질      0.600000
핏       1.000000
활용성     0.666667
Name: Pola_pred, dtype: float64